In [ ]:
#@title Install riffusion, etc.（riffusionなどをインストール）

!git clone https://github.com/hmartiro/riffusion-inference
%cd riffusion-inference
!pip install -q -r requirements.txt
!pip install -q diffusers[torch]==0.10.2 'transformers>=4.21.0' ftfy opencv-python einops safetensors

!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl

!wget -q https://github.com/chavinlo/riffusion-manipulation/raw/master/file2img.py
!wget -q https://github.com/chavinlo/riffusion-manipulation/raw/master/img2audio.py
!wget -q https://github.com/thx-pw/stable-diffusion-2-dreambooth/raw/main/gen_img_diffusers.py
!wget -q https://github.com/thx-pw/stable-diffusion-2-dreambooth/raw/main/model_util.py

In [ ]:
#@title Split mp3 into wav every 5 seconds and convert to spectrogram image（mp3を5秒ごとwavに分割、スペクトログラム画像に変換）
input_mp3 = "" #@param {type:"string"}

import os
import glob
from pydub import AudioSegment


os.makedirs('/content/input/wav_clip', exist_ok=True)
sound = AudioSegment.from_file(input_mp3, format="mp3")
sound = sound.set_channels(1)
sound_interval_time = 5000
for i, sound_index in enumerate(range(0, len(sound), sound_interval_time)):
  sound_interval = sound[sound_index:sound_index+sound_interval_time]
  sound_interval.export("/content/input/wav_clip/wav_clip_{0:04d}.wav".format(i), format="wav")


os.makedirs('/content/input/spectrogram', exist_ok=True)
files = sorted(glob.glob("/content/input/wav_clip/*.wav"))
for i, file in enumerate(files):
  output = "/content/input/spectrogram/spectrogram_{0:04d}.png".format(i)
  !python file2img.py -f {file} -o {output}

In [ ]:
#@title img2img of spectrogram image（スペクトログラム画像のimg2img）
prompt = "piano" #@param {type:"string"}
negative_prompt = "voice" #@param {type:"string"}
sampler = "ddim" #@param ["ddim","pndm","lms","euler","euler_a","heun","dpm_2","dpm_2_a","dpmsolver","dpmsolver++","dpmsingle","k_lms","k_euler","k_euler_a","k_dpm_2","k_dpm_2_a"]
steps = 50 #@param {type:"slider", min:1, max:150, step:1}
scale = 12.5 #@param {type:"slider", min:0, max:25, step:0.1}
strength = 0.3 #@param {type:"slider", min:0, max:1, step:0.01}

image_path = "/content/input/spectrogram"
outdir = '/content/output/spectrogram'
num_images = sum(os.path.isfile(os.path.join(image_path, name)) for name in os.listdir(image_path))
p = f"{prompt} --n {negative_prompt}"

/content/output/spectrogram
!python gen_img_diffusers.py --ckpt "riffusion/riffusion-model-v1" \
    --outdir {outdir} --xformers --fp16 --scale {scale} --sampler {sampler} --steps {steps} \
    --image_path {image_path} --strength {strength} \
    --prompt "{p}" \
    --batch_size 8 --images_per_prompt {num_images} --sequential_file_name

In [ ]:
#@title Convert spectrogram image to wav, merge and convert to output.mp3（スペクトログラム画像からwavに変換、結合してoutput.mp3に変換）
import os
import glob

os.makedirs('/content/output/wav_clip', exist_ok=True)
files = sorted(glob.glob("/content/output/spectrogram/*.png"))

for i, file in enumerate(files):
  output = "/content/output/wav_clip/wav_clip_{0:04d}.wav".format(i)
  !python img2audio.py -f {file} -o {output}

files = sorted(glob.glob("/content/output/wav_clip/*.wav"))
sound = None
for file in files:
  if sound is None:
    sound = AudioSegment.from_file(file, "wav")
  else:
    sound += AudioSegment.from_file(file, "wav")

sound.export("/content/output.mp3", format="mp3")